In [1]:
# Import dependencies
import torch
from PIL import Image
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Get Data
train = datasets.MNIST(root='data', download=True, train=True, transform=ToTensor())
dataset = DataLoader(train, 32)

In [3]:
# Image Classifier Neural Network
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6), 10)
        )

    def forward(self, x):
        return self.model(x)


In [4]:
# Instance of the Neural Network, loss, optimizer
clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [5]:
# Training flow
if __name__ == "__main__":
    for epoch in range(10): # train for 10 epochs
        for batch in dataset:
            X,y = batch
            X, y = X.to('cuda'), y.to('cuda')
            yhat = clf(X)
            loss = loss_fn(yhat, y)

            # Apply backprop
            opt.zero_grad()
            loss.backward()
            opt.step()

        print(f"Epoch:{epoch} loss is {loss.item()}")

    with open('model_state.pt', 'wb') as f:
        save(clf.state_dict(), f)

Epoch:0 loss is 0.009437127970159054
Epoch:1 loss is 0.00428627198562026
Epoch:2 loss is 0.03292979300022125
Epoch:3 loss is 0.00015004379383753985
Epoch:4 loss is 4.1183928260579705e-05
Epoch:5 loss is 2.8653952540480532e-05
Epoch:6 loss is 4.838952918362338e-06
Epoch:7 loss is 2.042753476416692e-05
Epoch:8 loss is 6.5226627157244366e-06
Epoch:9 loss is 1.713632116207009e-07
